In [1]:
# for similarity need to download larger english word library with the below in gitbash 
# python -m spacy download en_core_web_lg

In [2]:
import numpy as np
import pandas as pd
import spacy
import pyspark
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()

In [4]:
file = pd.read_csv("joined_data_v1.csv")

In [5]:
file = pd.DataFrame(file)
file.head()

,id,imdb_id,title,release_date,spoken_languages,genres1,genres2,genres3,genres4,genres5,...,director,poster_link,production_company,vote_average,vote_average (%),Internet_Movie_Database_rating,Rotten_Tomatoes_rating,Metacritic_rating,rating_average,writer
0,339408,tt4196450,The Birth of a Nation,9/9/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,NaN,NaN,NaN,NaN,...,Nate Parker,https://m.media-amazon.com/images/M/MV5BMTU5Mj...,Fox Searchlight Pictures,6.4,64%,64%,72%,69%,67%,"Nate Parker (screenplay by), Nate Parker (stor..."
1,339408,tt4196450,The Birth of a Nation,9/9/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,NaN,NaN,NaN,NaN,...,Nate Parker,https://m.media-amazon.com/images/M/MV5BMTU5Mj...,Fox Searchlight Pictures,6.4,64%,64%,72%,69%,67%,"Nate Parker (screenplay by), Nate Parker (stor..."
2,363676,tt3263904,Sully,9/7/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,History,NaN,NaN,NaN,...,Clint Eastwood,https://m.media-amazon.com/images/M/MV5BY2NmZD...,Warner Bros.,7.0,70%,75%,86%,74%,76%,"Todd Komarnicki, Chesley Sullenberger (based o..."
3,363676,tt3263904,Sully,9/7/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,History,NaN,NaN,NaN,...,Clint Eastwood,https://m.media-amazon.com/images/M/MV5BY2NmZD...,Warner Bros.,7.0,70%,75%,86%,74%,76%,"Todd Komarnicki, Chesley Sullenberger (based o..."
4,296524,tt1860357,Deepwater Horizon,9/29/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,Action,NaN,NaN,NaN,...,Peter Berg,https://m.media-amazon.com/images/M/MV5BOTEzND...,Lionsgate,6.5,65%,71%,83%,68%,72%,"Matthew Michael Carnahan (screenplay by), Matt..."


In [6]:
file.drop_duplicates(keep='first', inplace=True)

In [7]:
file.head()

,id,imdb_id,title,release_date,spoken_languages,genres1,genres2,genres3,genres4,genres5,...,director,poster_link,production_company,vote_average,vote_average (%),Internet_Movie_Database_rating,Rotten_Tomatoes_rating,Metacritic_rating,rating_average,writer
0,339408,tt4196450,The Birth of a Nation,9/9/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,NaN,NaN,NaN,NaN,...,Nate Parker,https://m.media-amazon.com/images/M/MV5BMTU5Mj...,Fox Searchlight Pictures,6.4,64%,64%,72%,69%,67%,"Nate Parker (screenplay by), Nate Parker (stor..."
2,363676,tt3263904,Sully,9/7/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,History,NaN,NaN,NaN,...,Clint Eastwood,https://m.media-amazon.com/images/M/MV5BY2NmZD...,Warner Bros.,7.0,70%,75%,86%,74%,76%,"Todd Komarnicki, Chesley Sullenberger (based o..."
4,296524,tt1860357,Deepwater Horizon,9/29/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,Action,NaN,NaN,NaN,...,Peter Berg,https://m.media-amazon.com/images/M/MV5BOTEzND...,Lionsgate,6.5,65%,71%,83%,68%,72%,"Matthew Michael Carnahan (screenplay by), Matt..."
6,283366,tt1935859,Miss Peregrine's Home for Peculiar Children,9/28/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,Fantasy,Adventure,NaN,NaN,...,Tim Burton,https://m.media-amazon.com/images/M/MV5BMTU0Nz...,20th Century Fox,6.5,65%,67%,65%,57%,64%,Ransom Riggs (based upon the novel written by)...
8,284564,tt3835080,31,9/15/2016,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Horror,Thriller,NaN,NaN,NaN,...,Rob Zombie,https://m.media-amazon.com/images/M/MV5BNTE0NT...,Bow and Arrow Entertainment,4.9,49%,51%,49%,35%,46%,Rob Zombie


In [8]:
overview_df2 = file[["imdb_id","title", "overview"]].copy()
overview_df2.head()

,imdb_id,title,overview
0,tt4196450,The Birth of a Nation,"Nat Turner, a former slave in America, leads a..."
2,tt3263904,Sully,"On 15 January 2009, the world witnessed the 'M..."
4,tt1860357,Deepwater Horizon,A story set on the offshore drilling rig Deepw...
6,tt1935859,Miss Peregrine's Home for Peculiar Children,A teenager finds himself transported to an isl...
8,tt3835080,31,ROB ZOMBIE'S S P O O K H A U S 31


In [9]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [10]:
sentenceData = spark.createDataFrame(overview_df2)
sentenceData.show() 

+---------+--------------------+--------------------+
|  imdb_id|               title|            overview|
+---------+--------------------+--------------------+
|tt4196450|The Birth of a Na...|Nat Turner, a for...|
|tt3263904|               Sully|On 15 January 200...|
|tt1860357|   Deepwater Horizon|A story set on th...|
|tt1935859|Miss Peregrine's ...|A teenager finds ...|
|tt3835080|                  31|ROB ZOMBIE'S  S P...|
|tt3774114|             Snowden|CIA employee Edwa...|
|tt1540011|         Blair Witch|Students on a cam...|
|tt2404435|The Magnificent S...|Seven gun men in ...|
|tt4520364|              Morgan|A corporate risk-...|
|tt1648190|      The Dark Tower|The last Gunsling...|
|tt4383594|          Nine Lives|A stuffy business...|
|tt4160708|       Don't Breathe|A group of teens ...|
|tt1386697|       Suicide Squad|From DC Comics co...|
|tt4302938|Kubo and the Two ...|Kubo mesmerizes t...|
|tt2638144|             Ben-Hur|A falsely accused...|
|tt2005151|            War D

In [11]:
tokenizer = Tokenizer(inputCol="overview", outputCol="words")
tokenizer

Tokenizer_a0dba5f024aa

In [12]:
# Transform and show DataFrame
tokenized = tokenizer.transform(sentenceData)
tokenized_2 = tokenizer.transform(sentenceData)
tokenized.show(truncate=False)

+---------+-------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|imdb_id  |title                                      |ove

In [13]:
# Instantiate Remover
remover = StopWordsRemover(inputCol="words", outputCol="stopwords")

In [14]:
# Transform and show data
tokenized = remover.transform(tokenized)
tokenized_2 = remover.transform(tokenized_2)
tokenized.show(truncate=False)

+---------+-------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------

In [15]:
tokenized_2.show(truncate=True)

+---------+--------------------+--------------------+--------------------+--------------------+
|  imdb_id|               title|            overview|               words|           stopwords|
+---------+--------------------+--------------------+--------------------+--------------------+
|tt4196450|The Birth of a Na...|Nat Turner, a for...|[nat, turner,, a,...|[nat, turner,, fo...|
|tt3263904|               Sully|On 15 January 200...|[on, 15, january,...|[15, january, 200...|
|tt1860357|   Deepwater Horizon|A story set on th...|[a, story, set, o...|[story, set, offs...|
|tt1935859|Miss Peregrine's ...|A teenager finds ...|[a, teenager, fin...|[teenager, finds,...|
|tt3835080|                  31|ROB ZOMBIE'S  S P...|[rob, zombie's, ,...|[rob, zombie's, ,...|
|tt3774114|             Snowden|CIA employee Edwa...|[cia, employee, e...|[cia, employee, e...|
|tt1540011|         Blair Witch|Students on a cam...|[students, on, a,...|[students, campin...|
|tt2404435|The Magnificent S...|Seven gu

In [16]:
tokenized.show(truncate=True)

+---------+--------------------+--------------------+--------------------+--------------------+
|  imdb_id|               title|            overview|               words|           stopwords|
+---------+--------------------+--------------------+--------------------+--------------------+
|tt4196450|The Birth of a Na...|Nat Turner, a for...|[nat, turner,, a,...|[nat, turner,, fo...|
|tt3263904|               Sully|On 15 January 200...|[on, 15, january,...|[15, january, 200...|
|tt1860357|   Deepwater Horizon|A story set on th...|[a, story, set, o...|[story, set, offs...|
|tt1935859|Miss Peregrine's ...|A teenager finds ...|[a, teenager, fin...|[teenager, finds,...|
|tt3835080|                  31|ROB ZOMBIE'S  S P...|[rob, zombie's, ,...|[rob, zombie's, ,...|
|tt3774114|             Snowden|CIA employee Edwa...|[cia, employee, e...|[cia, employee, e...|
|tt1540011|         Blair Witch|Students on a cam...|[students, on, a,...|[students, campin...|
|tt2404435|The Magnificent S...|Seven gu

# Starting Spacy Similarity code

In [18]:
# Load English tokenizer, tagger, parser, NER and word vectors
#MAKE SURE THIS IS LARGE LIBRARY

nlp = spacy.load("en_core_web_lg")
import string
punctuations = string.punctuation


## adding columns for similarities

In [19]:
from pyspark.sql.functions import lit

In [20]:
tokenized.show()

+---------+--------------------+--------------------+--------------------+--------------------+
|  imdb_id|               title|            overview|               words|           stopwords|
+---------+--------------------+--------------------+--------------------+--------------------+
|tt4196450|The Birth of a Na...|Nat Turner, a for...|[nat, turner,, a,...|[nat, turner,, fo...|
|tt3263904|               Sully|On 15 January 200...|[on, 15, january,...|[15, january, 200...|
|tt1860357|   Deepwater Horizon|A story set on th...|[a, story, set, o...|[story, set, offs...|
|tt1935859|Miss Peregrine's ...|A teenager finds ...|[a, teenager, fin...|[teenager, finds,...|
|tt3835080|                  31|ROB ZOMBIE'S  S P...|[rob, zombie's, ,...|[rob, zombie's, ,...|
|tt3774114|             Snowden|CIA employee Edwa...|[cia, employee, e...|[cia, employee, e...|
|tt1540011|         Blair Witch|Students on a cam...|[students, on, a,...|[students, campin...|
|tt2404435|The Magnificent S...|Seven gu

In [21]:
tokenized_2.show()

+---------+--------------------+--------------------+--------------------+--------------------+
|  imdb_id|               title|            overview|               words|           stopwords|
+---------+--------------------+--------------------+--------------------+--------------------+
|tt4196450|The Birth of a Na...|Nat Turner, a for...|[nat, turner,, a,...|[nat, turner,, fo...|
|tt3263904|               Sully|On 15 January 200...|[on, 15, january,...|[15, january, 200...|
|tt1860357|   Deepwater Horizon|A story set on th...|[a, story, set, o...|[story, set, offs...|
|tt1935859|Miss Peregrine's ...|A teenager finds ...|[a, teenager, fin...|[teenager, finds,...|
|tt3835080|                  31|ROB ZOMBIE'S  S P...|[rob, zombie's, ,...|[rob, zombie's, ,...|
|tt3774114|             Snowden|CIA employee Edwa...|[cia, employee, e...|[cia, employee, e...|
|tt1540011|         Blair Witch|Students on a cam...|[students, on, a,...|[students, campin...|
|tt2404435|The Magnificent S...|Seven gu

In [38]:
#Code to compare each movie to the all movie and recommend top 3
ll = []
result = []
for m in tokenized_2.collect():
    main_row = str(m.stopwords)
    docA = nlp(main_row)
    for f in tokenized.collect():

        overview_row = str(f.stopwords)
        #print(overview_row)
        doc_row = nlp(overview_row)

        compare = docA.similarity(doc_row)
        ll.append({'imdb_id': f.imdb_id,'Title': f.title, 'Similarity': compare })
# sample = tokenized.withColumn('similarity', lit(str(ll)))
# ll   
    df_simi = pd.DataFrame(ll)
    df_simi.sort_values(by=['Similarity'], inplace=True, ascending=False)
    df_simi.reset_index(drop=True)
    top3_df = df_simi.iloc[1:4]
    result.append({'imdb_id': m.imdb_id,'Title': m.title, 'Recom1': top3_df.iloc[0,2],
                  'Recom2': top3_df.iloc[1,2], 'Recom3': top3_df.iloc[2,2]})
    print(result)
#     top3_df

[{'imdb_id': 'tt4196450', 'Title': 'The Birth of a Nation', 'Recom1': 'tt4669986', 'Recom2': 'tt1124037', 'Recom3': 'tt4846340'}]
[{'imdb_id': 'tt4196450', 'Title': 'The Birth of a Nation', 'Recom1': 'tt4669986', 'Recom2': 'tt1124037', 'Recom3': 'tt4846340'}, {'imdb_id': 'tt3263904', 'Title': 'Sully', 'Recom1': 'tt3263904', 'Recom2': 'tt1594972', 'Recom3': 'tt4972582'}]
[{'imdb_id': 'tt4196450', 'Title': 'The Birth of a Nation', 'Recom1': 'tt4669986', 'Recom2': 'tt1124037', 'Recom3': 'tt4846340'}, {'imdb_id': 'tt3263904', 'Title': 'Sully', 'Recom1': 'tt3263904', 'Recom2': 'tt1594972', 'Recom3': 'tt4972582'}, {'imdb_id': 'tt1860357', 'Title': 'Deepwater Horizon', 'Recom1': 'tt3263904', 'Recom2': 'tt1860357', 'Recom3': 'tt1594972'}]
[{'imdb_id': 'tt4196450', 'Title': 'The Birth of a Nation', 'Recom1': 'tt4669986', 'Recom2': 'tt1124037', 'Recom3': 'tt4846340'}, {'imdb_id': 'tt3263904', 'Title': 'Sully', 'Recom1': 'tt3263904', 'Recom2': 'tt1594972', 'Recom3': 'tt4972582'}, {'imdb_id': 'tt1

[{'imdb_id': 'tt4196450', 'Title': 'The Birth of a Nation', 'Recom1': 'tt4669986', 'Recom2': 'tt1124037', 'Recom3': 'tt4846340'}, {'imdb_id': 'tt3263904', 'Title': 'Sully', 'Recom1': 'tt3263904', 'Recom2': 'tt1594972', 'Recom3': 'tt4972582'}, {'imdb_id': 'tt1860357', 'Title': 'Deepwater Horizon', 'Recom1': 'tt3263904', 'Recom2': 'tt1860357', 'Recom3': 'tt1594972'}, {'imdb_id': 'tt1935859', 'Title': "Miss Peregrine's Home for Peculiar Children", 'Recom1': 'tt3263904', 'Recom2': 'tt1860357', 'Recom3': 'tt1935859'}, {'imdb_id': 'tt3835080', 'Title': '31', 'Recom1': 'tt1935859', 'Recom2': 'tt3835080', 'Recom3': 'tt1860357'}, {'imdb_id': 'tt3774114', 'Title': 'Snowden', 'Recom1': 'tt3263904', 'Recom2': 'tt1935859', 'Recom3': 'tt3774114'}, {'imdb_id': 'tt1540011', 'Title': 'Blair Witch', 'Recom1': 'tt3263904', 'Recom2': 'tt1540011', 'Recom3': 'tt3774114'}, {'imdb_id': 'tt2404435', 'Title': 'The Magnificent Seven', 'Recom1': 'tt3263904', 'Recom2': 'tt1935859', 'Recom3': 'tt1860357'}, {'imdb_i

[{'imdb_id': 'tt4196450', 'Title': 'The Birth of a Nation', 'Recom1': 'tt4669986', 'Recom2': 'tt1124037', 'Recom3': 'tt4846340'}, {'imdb_id': 'tt3263904', 'Title': 'Sully', 'Recom1': 'tt3263904', 'Recom2': 'tt1594972', 'Recom3': 'tt4972582'}, {'imdb_id': 'tt1860357', 'Title': 'Deepwater Horizon', 'Recom1': 'tt3263904', 'Recom2': 'tt1860357', 'Recom3': 'tt1594972'}, {'imdb_id': 'tt1935859', 'Title': "Miss Peregrine's Home for Peculiar Children", 'Recom1': 'tt3263904', 'Recom2': 'tt1860357', 'Recom3': 'tt1935859'}, {'imdb_id': 'tt3835080', 'Title': '31', 'Recom1': 'tt1935859', 'Recom2': 'tt3835080', 'Recom3': 'tt1860357'}, {'imdb_id': 'tt3774114', 'Title': 'Snowden', 'Recom1': 'tt3263904', 'Recom2': 'tt1935859', 'Recom3': 'tt3774114'}, {'imdb_id': 'tt1540011', 'Title': 'Blair Witch', 'Recom1': 'tt3263904', 'Recom2': 'tt1540011', 'Recom3': 'tt3774114'}, {'imdb_id': 'tt2404435', 'Title': 'The Magnificent Seven', 'Recom1': 'tt3263904', 'Recom2': 'tt1935859', 'Recom3': 'tt1860357'}, {'imdb_i

[{'imdb_id': 'tt4196450', 'Title': 'The Birth of a Nation', 'Recom1': 'tt4669986', 'Recom2': 'tt1124037', 'Recom3': 'tt4846340'}, {'imdb_id': 'tt3263904', 'Title': 'Sully', 'Recom1': 'tt3263904', 'Recom2': 'tt1594972', 'Recom3': 'tt4972582'}, {'imdb_id': 'tt1860357', 'Title': 'Deepwater Horizon', 'Recom1': 'tt3263904', 'Recom2': 'tt1860357', 'Recom3': 'tt1594972'}, {'imdb_id': 'tt1935859', 'Title': "Miss Peregrine's Home for Peculiar Children", 'Recom1': 'tt3263904', 'Recom2': 'tt1860357', 'Recom3': 'tt1935859'}, {'imdb_id': 'tt3835080', 'Title': '31', 'Recom1': 'tt1935859', 'Recom2': 'tt3835080', 'Recom3': 'tt1860357'}, {'imdb_id': 'tt3774114', 'Title': 'Snowden', 'Recom1': 'tt3263904', 'Recom2': 'tt1935859', 'Recom3': 'tt3774114'}, {'imdb_id': 'tt1540011', 'Title': 'Blair Witch', 'Recom1': 'tt3263904', 'Recom2': 'tt1540011', 'Recom3': 'tt3774114'}, {'imdb_id': 'tt2404435', 'Title': 'The Magnificent Seven', 'Recom1': 'tt3263904', 'Recom2': 'tt1935859', 'Recom3': 'tt1860357'}, {'imdb_i

[{'imdb_id': 'tt4196450', 'Title': 'The Birth of a Nation', 'Recom1': 'tt4669986', 'Recom2': 'tt1124037', 'Recom3': 'tt4846340'}, {'imdb_id': 'tt3263904', 'Title': 'Sully', 'Recom1': 'tt3263904', 'Recom2': 'tt1594972', 'Recom3': 'tt4972582'}, {'imdb_id': 'tt1860357', 'Title': 'Deepwater Horizon', 'Recom1': 'tt3263904', 'Recom2': 'tt1860357', 'Recom3': 'tt1594972'}, {'imdb_id': 'tt1935859', 'Title': "Miss Peregrine's Home for Peculiar Children", 'Recom1': 'tt3263904', 'Recom2': 'tt1860357', 'Recom3': 'tt1935859'}, {'imdb_id': 'tt3835080', 'Title': '31', 'Recom1': 'tt1935859', 'Recom2': 'tt3835080', 'Recom3': 'tt1860357'}, {'imdb_id': 'tt3774114', 'Title': 'Snowden', 'Recom1': 'tt3263904', 'Recom2': 'tt1935859', 'Recom3': 'tt3774114'}, {'imdb_id': 'tt1540011', 'Title': 'Blair Witch', 'Recom1': 'tt3263904', 'Recom2': 'tt1540011', 'Recom3': 'tt3774114'}, {'imdb_id': 'tt2404435', 'Title': 'The Magnificent Seven', 'Recom1': 'tt3263904', 'Recom2': 'tt1935859', 'Recom3': 'tt1860357'}, {'imdb_i

...

[{'imdb_id': 'tt4196450', 'Title': 'The Birth of a Nation', 'Recom1': 'tt4669986', 'Recom2': 'tt1124037', 'Recom3': 'tt4846340'}, {'imdb_id': 'tt3263904', 'Title': 'Sully', 'Recom1': 'tt3263904', 'Recom2': 'tt1594972', 'Recom3': 'tt4972582'}, {'imdb_id': 'tt1860357', 'Title': 'Deepwater Horizon', 'Recom1': 'tt3263904', 'Recom2': 'tt1860357', 'Recom3': 'tt1594972'}, {'imdb_id': 'tt1935859', 'Title': "Miss Peregrine's Home for Peculiar Children", 'Recom1': 'tt3263904', 'Recom2': 'tt1860357', 'Recom3': 'tt1935859'}, {'imdb_id': 'tt3835080', 'Title': '31', 'Recom1': 'tt1935859', 'Recom2': 'tt3835080', 'Recom3': 'tt1860357'}, {'imdb_id': 'tt3774114', 'Title': 'Snowden', 'Recom1': 'tt3263904', 'Recom2': 'tt1935859', 'Recom3': 'tt3774114'}, {'imdb_id': 'tt1540011', 'Title': 'Blair Witch', 'Recom1': 'tt3263904', 'Recom2': 'tt1540011', 'Recom3': 'tt3774114'}, {'imdb_id': 'tt2404435', 'Title': 'The Magnificent Seven', 'Recom1': 'tt3263904', 'Recom2': 'tt1935859', 'Recom3': 'tt1860357'}, {'imdb_i

[{'imdb_id': 'tt4196450', 'Title': 'The Birth of a Nation', 'Recom1': 'tt4669986', 'Recom2': 'tt1124037', 'Recom3': 'tt4846340'}, {'imdb_id': 'tt3263904', 'Title': 'Sully', 'Recom1': 'tt3263904', 'Recom2': 'tt1594972', 'Recom3': 'tt4972582'}, {'imdb_id': 'tt1860357', 'Title': 'Deepwater Horizon', 'Recom1': 'tt3263904', 'Recom2': 'tt1860357', 'Recom3': 'tt1594972'}, {'imdb_id': 'tt1935859', 'Title': "Miss Peregrine's Home for Peculiar Children", 'Recom1': 'tt3263904', 'Recom2': 'tt1860357', 'Recom3': 'tt1935859'}, {'imdb_id': 'tt3835080', 'Title': '31', 'Recom1': 'tt1935859', 'Recom2': 'tt3835080', 'Recom3': 'tt1860357'}, {'imdb_id': 'tt3774114', 'Title': 'Snowden', 'Recom1': 'tt3263904', 'Recom2': 'tt1935859', 'Recom3': 'tt3774114'}, {'imdb_id': 'tt1540011', 'Title': 'Blair Witch', 'Recom1': 'tt3263904', 'Recom2': 'tt1540011', 'Recom3': 'tt3774114'}, {'imdb_id': 'tt2404435', 'Title': 'The Magnificent Seven', 'Recom1': 'tt3263904', 'Recom2': 'tt1935859', 'Recom3': 'tt1860357'}, {'imdb_i

[{'imdb_id': 'tt4196450', 'Title': 'The Birth of a Nation', 'Recom1': 'tt4669986', 'Recom2': 'tt1124037', 'Recom3': 'tt4846340'}, {'imdb_id': 'tt3263904', 'Title': 'Sully', 'Recom1': 'tt3263904', 'Recom2': 'tt1594972', 'Recom3': 'tt4972582'}, {'imdb_id': 'tt1860357', 'Title': 'Deepwater Horizon', 'Recom1': 'tt3263904', 'Recom2': 'tt1860357', 'Recom3': 'tt1594972'}, {'imdb_id': 'tt1935859', 'Title': "Miss Peregrine's Home for Peculiar Children", 'Recom1': 'tt3263904', 'Recom2': 'tt1860357', 'Recom3': 'tt1935859'}, {'imdb_id': 'tt3835080', 'Title': '31', 'Recom1': 'tt1935859', 'Recom2': 'tt3835080', 'Recom3': 'tt1860357'}, {'imdb_id': 'tt3774114', 'Title': 'Snowden', 'Recom1': 'tt3263904', 'Recom2': 'tt1935859', 'Recom3': 'tt3774114'}, {'imdb_id': 'tt1540011', 'Title': 'Blair Witch', 'Recom1': 'tt3263904', 'Recom2': 'tt1540011', 'Recom3': 'tt3774114'}, {'imdb_id': 'tt2404435', 'Title': 'The Magnificent Seven', 'Recom1': 'tt3263904', 'Recom2': 'tt1935859', 'Recom3': 'tt1860357'}, {'imdb_i

[{'imdb_id': 'tt4196450', 'Title': 'The Birth of a Nation', 'Recom1': 'tt4669986', 'Recom2': 'tt1124037', 'Recom3': 'tt4846340'}, {'imdb_id': 'tt3263904', 'Title': 'Sully', 'Recom1': 'tt3263904', 'Recom2': 'tt1594972', 'Recom3': 'tt4972582'}, {'imdb_id': 'tt1860357', 'Title': 'Deepwater Horizon', 'Recom1': 'tt3263904', 'Recom2': 'tt1860357', 'Recom3': 'tt1594972'}, {'imdb_id': 'tt1935859', 'Title': "Miss Peregrine's Home for Peculiar Children", 'Recom1': 'tt3263904', 'Recom2': 'tt1860357', 'Recom3': 'tt1935859'}, {'imdb_id': 'tt3835080', 'Title': '31', 'Recom1': 'tt1935859', 'Recom2': 'tt3835080', 'Recom3': 'tt1860357'}, {'imdb_id': 'tt3774114', 'Title': 'Snowden', 'Recom1': 'tt3263904', 'Recom2': 'tt1935859', 'Recom3': 'tt3774114'}, {'imdb_id': 'tt1540011', 'Title': 'Blair Witch', 'Recom1': 'tt3263904', 'Recom2': 'tt1540011', 'Recom3': 'tt3774114'}, {'imdb_id': 'tt2404435', 'Title': 'The Magnificent Seven', 'Recom1': 'tt3263904', 'Recom2': 'tt1935859', 'Recom3': 'tt1860357'}, {'imdb_i

KeyboardInterrupt: 

# Importing recommandation to csv

In [41]:
df_recomm = pd.DataFrame(result)
df_recomm.head()

,Recom1,Recom2,Recom3,Title,imdb_id
0,tt4669986,tt1124037,tt4846340,The Birth of a Nation,tt4196450
1,tt3263904,tt1594972,tt4972582,Sully,tt3263904
2,tt3263904,tt1860357,tt1594972,Deepwater Horizon,tt1860357
3,tt3263904,tt1860357,tt1935859,Miss Peregrine's Home for Peculiar Children,tt1935859
4,tt1935859,tt3835080,tt1860357,31,tt3835080


In [43]:
df_recomm.to_csv('/Users/ericH/Desktop/moviesDashboard/raw/recom.csv', index = None, header=True)